<a href="https://colab.research.google.com/github/Otabek-Rizayev/AI-Models/blob/main/diabet_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url = "https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/refs/heads/main/diabetes.csv"
df = pd.read_csv(url)
df.sample(10)

In [ ]:
df.shape

In [ ]:
df['Outcome'].value_counts()

In [ ]:
corr_matrix = df.corr().abs()
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
# Feautureslarni predictorga nisbatan korrelatsiyasi
corr = df.corrwith(df['Outcome']).abs().sort_values(ascending=False)
corr

In [ ]:
# Eng yuqori korrelatsiyaga ega 5 ta ustunni ajratish
droped_col = corr.head(5)
X = df.drop(columns=[col for col in df.columns if col not in droped_col], axis=1)
y = df['Outcome']
X

In [ ]:
# Ma'lumotlarni standartizatsiyalash
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# KNNga eng yuqori aniqlikdagi K parameterini kiritish
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=12)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1) # k-ni qiymati
knn.fit(X_train, y_train)

In [ ]:
# Modelni ishga tushirish
y_predict = knn.predict(X_test)

In [ ]:
# Jaccard Index orqali baholash
from sklearn.metrics import jaccard_score
jaccard_score(y_test, y_predict)

In [ ]:
# Confusion Matrix orqali baholash
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)
plt.show()

In [ ]:
# Precision, recall, F1 orqali baholash
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict))

In [ ]:
# Precision, recall, F1 bitta-bitta baholash
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
precision = precision_score(y_test, y_predict)
recall = recall_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)
accuracy = accuracy_score(y_test, y_predict)
print(f"{precision=}\n{recall=}\n{f1=}\n{accuracy=}")

In [ ]:
# Cross validation orqali baholash
from sklearn.model_selection import cross_val_predict
predict = cross_val_predict(estimator = knn, X = X, y = y, cv = 5)
from sklearn.metrics import classification_report
print("Classification Report: \n",classification_report(y, predict))

In [ ]:
# Eng yaxshi K parameterini topish (manual)
f1 = []
for k in range(1,25):
    knn = KNeighborsClassifier(n_neighbors=k) # k-ni qiymati
    knn.fit(X_train, y_train)
    y_predict = knn.predict(X_test)
    f1.append(f1_score(y_test, y_predict))

plt.figure(figsize=(10,6))
plt.plot(range(1,25),f1)
plt.xticks(range(1,25))
plt.grid()
plt.show()

In [ ]:
# Eng yaxshi K parameterini topish
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors': np.arange(1, 25)}
knn_gscv = GridSearchCV(knn, param_grid, cv=5)
knn_gscv.fit(X, y)

In [ ]:
# Barcha kategoriyalarni ko'rish
knn_gscv.cv_results_['rank_test_score']

In [ ]:
# Eng yaxshi parameterlarni bilish
knn_gscv.best_params_

In [ ]:
# Eng yaxshi qiymat
knn_gscv.best_score_

In [ ]:
# Grafikda ko'rish
plt.figure(figsize=(10,6))
plt.plot(param_grid['n_neighbors'], knn_gscv.cv_results_['rank_test_score'])
plt.xticks(param_grid['n_neighbors'])
plt.xlabel("k")
plt.ylabel("Xatolik reytingi")
plt.grid()
plt.show()